In [114]:
import requests
import pandas as pd
import openpyxl
import tqdm

### Parsing functions

In [128]:
# get group ids by a search query
def group_byquery(q, count, api_key):
    api_endpoint = f'https://api.vk.com/method/groups.search?q={q}&type=group&count={count}&f&v=5.52&access_token={api_key}'
    r = requests.get(api_endpoint)
    data = r.json()
    ids = [data['response']['items'][x]['id'] for x in range(0, count)]
    return ids

# get group posts text
def get_data(group_id, count, api_key):
    text = []
    api_endpoint = f'https://api.vk.com/method/wall.get?owner_id=-{str(group_id)}&count={count}&filter=owner&f&v=5.52&access_token={api_key}'
    r = requests.get(api_endpoint)
    post_data = r.json()
    try:
         text = [post_data['response']['items'][n]['text'] for n in range(0,count)]
         text = ' '.join(text)
    except:
        pass
    return text
    
# get number of group followers
def member_count(group_id, api_key):
    api_endpoint_count = f'https://api.vk.com/method/groups.getById?group_ids={group_id}&fields=members_count&v=5.52&access_token={api_key}'
    k = requests.get(api_endpoint_count)
    count_data = k.json()
    try:
        members = count_data['response'][0]['members_count']
    except:
        members = 0
        pass
    return members
# get group member ids
def get_member_ids(group_id, api_key):
    api_endpoint = f'https://api.vk.com/method/groups.getMembers?group_id={group_id}&sort=id_asc&v=5.52&access_token={api_key}'
    r = requests.get(api_endpoint)
    data = r.json()
    try:
        member_ids = data['response']['items']
    except:
        member_ids = 0
        pass
    return member_ids

In [ ]:
count = 300 # namber of groups to parse
q = 'музыка' #книги наука
api_key = '' # here is your vk api

In [174]:
ids = group_byquery(q, count, api_key)

In [175]:
count = 100 #number of posts to parse
final_df = pd.DataFrame(ids, columns=["ids"]).set_index('ids', drop=True)
res_text = []
members_count = []
member_ids = []
for group_id in tqdm.tqdm(final_df.index):
     res_text.append(get_data(group_id, count, api_key))
     members_count.append(member_count(group_id, api_key))
     member_ids.append(get_member_ids(group_id, api_key))
final_df['members_count'] = members_count
final_df['text'] = res_text
final_df['member_ids'] = member_ids

100%|██████████| 300/300 [03:25<00:00,  1.46it/s]


In [177]:
final_df 

,members_count,text,member_ids
ids,,,
27737784,328785,Какова роль фундаментальной науки в борьбе с п...,"[48, 348, 485, 575, 628, 708, 837, 905, 976, 1..."
190320587,4934,Вниманию детей и их родителей! \n \nПроводится...,"[423, 804, 1725, 8513, 13680, 15110, 18541, 21..."
119331,35199,[],"[1586, 2051, 2925, 3058, 3406, 3473, 4049, 414..."
153252740,5329,🆘 ПОДПИШИТЕ ПЕТИЦИЮ ПРОТИВ ПРИНЯТИЯ В РОССИИ Н...,"[2774, 3186, 4990, 37082, 53556, 54471, 85322,..."
20393607,26788,Пятизвездочный 🐄 коровник || Расскажу @ЗА ПАРУ...,"[3450, 3721, 4646, 5280, 5511, 5846, 6376, 641..."
...,...,...,...
129392181,588,[],"[817333, 2322848, 2705487, 3340259, 3597780, 3..."
100031872,0,[],0
128486104,0,[],0


In [178]:
final_df.to_csv('science2.csv') #saving data